In [3]:
import numpy as np
import pandas as pd

# Для примера возьмём train_test_split, LabelEncoder, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

np.random.seed(42)

# Количество записей в наборе данных
N = 1000

# 1) Время суток (0-23)
time_of_day = np.random.randint(0, 24, size=N)

# 2) Скорость (km/h)
speed = np.random.uniform(20, 120, size=N)

# 3) Погодные условия
weather_conditions = ["Солнечно", "Облачно", "Дождь", "Снег"]
weather = np.random.choice(weather_conditions, size=N, p=[0.4, 0.3, 0.2, 0.1])

# 4) Плотность трафика (целевая переменная)
traffic = []
for t in time_of_day:
    if 7 <= t < 10 or 17 <= t < 20:
        traffic.append(np.random.choice(["средняя", "высокая"], p=[0.3, 0.7]))
    else:
        traffic.append(np.random.choice(["низкая", "средняя"], p=[0.6, 0.4]))

# Формируем исходный DataFrame
df = pd.DataFrame({
    "time_of_day": time_of_day,
    "speed_km_h": speed,
    "weather": weather,
    "traffic_density": traffic
})

# Посмотрим на первые строки
df.head()


,time_of_day,speed_km_h,weather,traffic_density
0,6,31.483682,Солнечно,средняя
1,19,81.062004,Солнечно,высокая
2,14,48.863055,Солнечно,средняя
3,10,78.123822,Солнечно,низкая
4,7,35.436272,Снег,высокая


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Предположим, что df - наш исходный DataFrame
# df = pd.DataFrame({...})

# Разделяем данные. Параметр test_size=0.2 означает,
# что 20% данных пойдут в тестовую выборку, а остальные 80% — в обучающую.
# random_state для воспроизводимости.
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

print("Размер обучающей выборки:", train_df.shape)
print("Размер тестовой выборки:", test_df.shape)


Размер обучающей выборки: (800, 4)
Размер тестовой выборки: (200, 4)


In [5]:
# X - набор признаков
X = df[["time_of_day", "speed_km_h", "weather"]]
# y - целевая переменная (traffic_density)
y = df["traffic_density"]


In [7]:
# Вместо
# X = df[["time_of_day", "speed_km_h", "weather"]]
# используем
X = df[["time_of_day", "speed_km_h", "weather"]].copy()

# Теперь можно спокойно менять X
X["weather"] = weather_encoder.fit_transform(X["weather"])


In [8]:
# Преобразуем weather (X) в числовые значения
weather_encoder = LabelEncoder()
X["weather"] = weather_encoder.fit_transform(X["weather"])

# Аналогично для y (traffic_density)
traffic_encoder = LabelEncoder()
y_encoded = traffic_encoder.fit_transform(y)  # y_encoded - это числовой аналог y


In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X,            # Признаки
    y_encoded,    # Целевая переменная (числовая)
    test_size=0.2,
    random_state=42
)

print("Размер X_train:", X_train.shape)
print("Размер X_test:", X_test.shape)


Размер X_train: (800, 3)
Размер X_test: (200, 3)


In [11]:
# Создаём модель
model = RandomForestClassifier(random_state=42)

# Обучаем (fit) модель на обучающих данных
model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [12]:
# Предсказания модели
y_pred = model.predict(X_test)

# Точность (accuracy)
acc = accuracy_score(y_test, y_pred)
print("Accuracy:", acc)

# Подробнее – отчёт о классификации (precision, recall, f1-score и т.д.)
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=traffic_encoder.classes_))


Accuracy: 0.53
Classification Report:
              precision    recall  f1-score   support

     высокая       0.73      0.63      0.68        38
      низкая       0.54      0.72      0.61        82
     средняя       0.40      0.29      0.34        80

    accuracy                           0.53       200
   macro avg       0.56      0.55      0.54       200
weighted avg       0.52      0.53      0.51       200



In [14]:
def predict_traffic():
    # 1) Ввод данных от пользователя
    try:
        user_time_of_day = int(input("Введите час суток (0-23): "))
        user_speed = float(input("Введите скорость движения (км/ч): "))
        user_weather = input("Введите погоду (Солнечно, Облачно, Дождь, Снег): ")
        
        # 2) Проверка/приведение данных к нужному формату
        # Убедимся, что погода написана корректно (с учётом того, что мы использовали LabelEncoder).
        # Если пользователь введёт неверно (например, "солнечно" с маленькой буквы), то будет ошибка.
        # Для простоты можно либо привести всё к нужному регистру, либо добавить логику обработки ошибок.
        
        # Преобразуем weather
        # Если такого значения нет в обученном энкодере, будет ошибка. 
        # Либо вы можете вручную обработать "неизвестные" значения.
        encoded_weather = weather_encoder.transform([user_weather])
        
        # 3) Создадим DataFrame с одним наблюдением
        X_input = pd.DataFrame({
            "time_of_day": [user_time_of_day],
            "speed_km_h": [user_speed],
            "weather": encoded_weather
        })
        
        # 4) Предсказываем
        pred = model.predict(X_input)
        
        # 5) Обратное преобразование в категории
        predicted_traffic = traffic_encoder.inverse_transform(pred)[0]
        
        # 6) Вывод результата
        print(f"\nПрогнозируемая плотность трафика: {predicted_traffic}\n")
    
    except ValueError as e:
        print(f"Ошибка ввода: {e}")
    except Exception as e:
        print(f"Произошла ошибка: {e}")

# Запустим цикл предсказания (для примера — можно вводить несколько раз)
while True:
    print("=== Предсказание плотности трафика ===")
    predict_traffic()
    cont = input("Хотите продолжить? (y/n): ").strip().lower()
    if cont != 'y':
        print("Выход из программы.")
        break

=== Предсказание плотности трафика ===


Введите час суток (0-23):  12
Введите скорость движения (км/ч):  80
Введите погоду (Солнечно, Облачно, Дождь, Снег):  1



Прогнозируемая плотность трафика: низкая



Хотите продолжить? (y/n):  n


Выход из программы.
